JUST RUN THIS ON GOOGLE COLAB, OKAY? AND MAKE SURE TO USE GPU.

1. clone the repo and install all the requirements

In [ ]:
#@title Install TGWUI
silent = True # @param {type:"boolean"}
logins = True # @param {type:"boolean"}
Model = ""
%cd /content/
if silent:
  !cd /content/ ; git clone -q https://github.com/TheBlokeAI/AIScripts.git -q
  !pip install GPUtil -q -q -q --progress-bar off > /dev/null
  !pip install huggingface_hub -q -q -q --progress-bar off > /dev/null
  !pip install hf_transfer -q -q -q --progress-bar off > /dev/null
  if logins:
    from huggingface_hub import login
    login("ur hf key here", add_to_git_credential=True)
  import torch
  from pathlib import Path
  import time
  import threading
  import GPUtil
  from tabulate import tabulate
  !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb -q -nv
  !DEBIAN_FRONTEND=noninteractive apt-get install -qq ./cloudflared-linux-amd64.deb aria2 < /dev/null > /dev/null
  !rm cloudflared-linux-amd64.deb

  !rm -r /content/sample_data
  if Path.cwd().name != 'text-generation-webui':
    print('\033[1;32;1m\nInstalling...\n\033[0;37;0m')

    !git clone -q -b dev https://github.com/oobabooga/text-generation-webui
    !ln -s text-generation-webui/models .
    %cd text-generation-webui

    torver = torch.__version__
    print(f"TORCH: {torver}")
    is_cuda118 = '+cu118' in torver  # 2.1.0+cu118
    is_cuda117 = '+cu117' in torver  # 2.0.1+cu117

    textgen_requirements = open('requirements.txt').read().splitlines()
    if is_cuda117:
        textgen_requirements = [req.replace('+cu121', '+cu117').replace('+cu122', '+cu117').replace('torch2.1', 'torch2.0') for req in textgen_requirements]
    elif is_cuda118:
        textgen_requirements = [req.replace('+cu121', '+cu118').replace('+cu122', '+cu118') for req in textgen_requirements]
    with open('temp_requirements.txt', 'w') as file:
        file.write('\n'.join(textgen_requirements))

    !pip install -r extensions/openai/requirements.txt --upgrade -q -q -q --progress-bar off --no-warn-conflicts > /dev/null
    !pip install -r temp_requirements.txt --upgrade -q -q -q --progress-bar off --no-warn-conflicts > /dev/null

    try:
      import flash_attn
    except:
      !pip uninstall -y flash_attn

  print('\033[1;32;1m\nDone!\n\033[0;37;0m')
else:
  !cd /content/ ; git clone -q https://github.com/TheBlokeAI/AIScripts.git
  !pip install GPUtil
  !pip install huggingface_hub
  !pip install hf_transfer
  if logins:
    from huggingface_hub import login
    login("ur hf key here", add_to_git_credential=True)
  import torch
  from pathlib import Path
  import time
  import threading
  import GPUtil
  from tabulate import tabulate
  !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb -q -nv
  !apt install ./cloudflared-linux-amd64.deb aria2
  !rm cloudflared-linux-amd64.deb

  !rm -r /content/sample_data
  if Path.cwd().name != 'text-generation-webui':
    print('\033[1;32;1m\nInstalling...\n\033[0;37;0m')

    !git clone -b dev https://github.com/oobabooga/text-generation-webui
    !ln -s text-generation-webui/models .
    %cd text-generation-webui

    torver = torch.__version__
    print(f"TORCH: {torver}")
    is_cuda118 = '+cu118' in torver  # 2.1.0+cu118
    is_cuda117 = '+cu117' in torver  # 2.0.1+cu117

    textgen_requirements = open('requirements.txt').read().splitlines()
    if is_cuda117:
        textgen_requirements = [req.replace('+cu121', '+cu117').replace('+cu122', '+cu117').replace('torch2.1', 'torch2.0') for req in textgen_requirements]
    elif is_cuda118:
        textgen_requirements = [req.replace('+cu121', '+cu118').replace('+cu122', '+cu118') for req in textgen_requirements]
    with open('temp_requirements.txt', 'w') as file:
        file.write('\n'.join(textgen_requirements))

    !pip install -r extensions/openai/requirements.txt --upgrade --no-warn-conflicts
    !pip install -r temp_requirements.txt --upgrade --no-warn-conflicts

    try:
      import flash_attn
    except:
      !pip uninstall -y flash_attn

  print('\033[1;32;1m\nDone!\n\033[0;37;0m')

Download da model :>

download model sendiri, pilih di https://huggingface.co/models (hanya untuk text generation)

In [ ]:
#@title Model
#@markdown If unsure about the branch, write "main" or leave it blank.

branch = "main" # @param ["main", "6bit", "gptq-4bit-32g-actorder_True", "gptq-4bit-128g-actorder_True"] {allow-input: true}

if branch == "":
  branch = "main"

Models = "R136a1/TimeCrystal-l2-13B-exl2" # @param ["R136a1/MythoMax-L2-13B-exl2", "TheBloke/Euryale-1.3-L2-70B-GPTQ", "TheBloke/Xwin-LM-70B-V0.1-GPTQ", "R136a1/l2-13b-thespurral-v1-exl2", "R136a1/TimeCrystal-l2-13B-exl2", "R136a1/Noromaid-20b-v0.1.1-exl2", "R136a1/TimeLess-20B-exl2", "R136a1/TimeMax-20B-exl2"] {allow-input: true}
Model = Models.split("/")[-1]

!pip show exllamav2

%cd /content/AIScripts/
!HF_HUB_ENABLE_HF_TRANSFER=1

!python hub_download.py --symlinks false $Models "/content/models/"$Model --branch $branch

Gass ga si? gass lah (kalau tidak punya tunnel cloudflarenya, pakai mode "no tunnel + no model")

"no model" artinya tidak ada model yang di load, jadi load sendiri dari gradio

In [ ]:
#@title GASS
Token = "isi pake Tunnel ID sendiri" #@param {type:"string"}
Loader = "Exllamav2_hf" # @param ["Exllamav2_hf", "Exllamav2"]
mode = "no model" # @param ["gas", "no model", "no tunnel + no model"]

# monitor GPU stats
def gpuinfo():
    i = 0
    while True:
        # GPU information
        print("="*40, "GPU Details", "="*40)
        gpus = GPUtil.getGPUs()
        list_gpus = []
        for gpu in gpus:
            # get the GPU id
            gpu_id = gpu.id
            # name of GPU
            gpu_name = gpu.name
            # get % percentage of GPU usage of that GPU
            gpu_load = f"{gpu.load*100}%"
            # get free memory in MB format
            gpu_free_memory = f"{gpu.memoryFree}MB"
            # get used memory
            gpu_used_memory = f"{gpu.memoryUsed}MB"
            # get total memory
            gpu_total_memory = f"{gpu.memoryTotal}MB"
            # get GPU temperature in Celsius
            gpu_temperature = f"{gpu.temperature} °C"
            gpu_uuid = gpu.uuid
            list_gpus.append((
                gpu_id, gpu_name, gpu_load, gpu_free_memory, gpu_used_memory,
                gpu_total_memory, gpu_temperature, gpu_uuid
            ))

        print(tabulate(list_gpus, headers=("id", "name", "load", "free memory", "used memory", "total memory",
                                        "temperature", "uuid")))
        print("\n"*2)
        time.sleep(600)
thread1 = threading.Thread(target=gpuinfo)
thread1.start()

server1 = f"cd /content/text-generation-webui ;python server.py --extensions openai --gradio-auth user:admin"
server2 = f"cd /content/text-generation-webui ;python server.py --extensions openai --gradio-auth user:admin --public-api --share"
server = f"cd /content/text-generation-webui ;python server.py --model {Model} --extensions openai --loader {Loader} --max_seq_len 4096 --compress_pos_emb 1 --gradio-auth user:admin"
if mode == "no tunnel + no model":
  !$server2
if mode == "no model":
  !cloudflared service install $Token
  !$server1
else:
  !!cloudflared service install $Token
  !$server